In [1]:
import os
import gc
import cv2
import math
import copy
import time
import random
from glob import glob
from matplotlib import pyplot as plt
import pyvips

# For data manipulation
import numpy as np
import pandas as pd
from PIL import Image

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
from pathlib import Path
from joblib import Parallel, delayed

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm
import segmentation_models_pytorch as smp

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class CFG:
    model_name = "tf_efficientnetv2_s_in21ft1k"
    selected_folds = [0, 1, 2, 3, 4]
    img_size = 512
    # batch_size and epochs
    batch_size = 8
    num_workers = 4
    label_dict = {0: "CC", 1: "EC", 2: "HGSC", 3: "LGSC", 4: "MC", 5:"Other"}
    num_classes = 5
    
    seg_model_name = "efficientnet-b0"
    seg_model_weight = "exp-seg/seg-fold-0.pth"
    
    clc_model_name = "tf_efficientnetv2_s_in21ft1k"
    clc_model_weight = "/kaggle/input/ubc-ocean/exp06-fold-0-acc0.77.pth"

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

In [4]:
Debug= True
data_dir = Path("dataset")
test_df = pd.read_csv(data_dir / "test.csv")
test_df["is_tma"] =test_df["image_width"] < 6000
img_path = "test_images"

In [5]:
if Debug:
    train_df = pd.read_csv(data_dir / "train.csv" )
    test_df = train_df[:20]

In [6]:
target_path = Path("test_tiles")
target_path.mkdir(exist_ok=True)

In [7]:
def process_image(row, data_dir, target_path):
    tma = row["is_tma"]
    image_id = row["image_id"]
    
    img_path = data_dir / "train_images" / f"{row.image_id}.png"
    


    new_size = (512, 512)
    tile_size = 2048
    if ~tma:
        img = pyvips.Image.new_from_file(img_path)
        height = img.height
        width = img.width
        # Iterate over the image in tiles
        for y in range(0, height, tile_size):
            for x in range(0, width, tile_size):
                # Extract a tile from the image and mask
                img_tile = img.crop(
                    x, y, min(tile_size, width - x), min(tile_size, height - y)
                ).numpy()

                gray = cv2.cvtColor(img_tile, cv2.COLOR_BGR2GRAY)
                _, binary_image = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

                # black area ratio threshold start from here
                black_pixels = np.count_nonzero(binary_image == 0)
                total_pixels = np.prod(binary_image.shape[:2])
                black_area_ratio = black_pixels / total_pixels


                if black_area_ratio > 0.3:
#                     print(f"black_area_ratio: {black_area_ratio}")
                    continue

                tile_save_path = target_path / f"{image_id}_tile_{x}_{y}.png"
                Image.fromarray(img_tile).resize(new_size, Image.LANCZOS).save(
                            tile_save_path
                        )
        

    else:
        img = Image.open(img_path)
        target_path = target_path / f"{image_id}_tile.png"
        Image.fromarray(img_tile).resize(new_size, Image.LANCZOS).save(
                    target_path
                )

In [11]:
num_processes = 4
_ = Parallel(n_jobs=num_processes, backend="multiprocessing")(
    delayed(process_image)(row, data_dir, target_path)
    for _, row in tqdm(test_df.iterrows(), total=len(test_df))
)

100%|██████████| 20/20 [09:17<00:00, 27.88s/it]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
tile_files = glob(str(target_path / "*.png"))
tile_df = pd.DataFrame(tile_files, columns=["tile_path"])
tile_df["image_id"] =   tile_df["tile_path"].apply(lambda x: x.split("/")[-1].split("_")[0])


In [9]:
def get_transforms():
    return A.Compose(
        [
            A.Resize(CFG.img_size, CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0,
            ),
            ToTensorV2(),
        ],
        p=1.0,)

In [10]:
class UBCSegDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.img_paths = df["tile_path"].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        
        img_path = self.img_paths[index]
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            "image": img,
        }
    
def seg_infer(test_loader, model):
    
    mask_ratio = []
    for index, data in enumerate(test_loader):
        images = data["image"].to(device)
        batch_size = images.size(0) 
        with torch.no_grad():
            outputs = model(images)

        probs = torch.sigmoid(outputs)
        probs = probs.detach().cpu().numpy()

        threshold = 0.33
        binary_masks = (probs > threshold).astype(int)
        
         
        for _index in range(batch_size):
            true_pixel_ratio = np.count_nonzero(binary_masks[_index]) /(512*512)
            mask_ratio.append(true_pixel_ratio)
    #     for index, image_id in enumerate(image_ids):
    #         _image_ids.append(str(image_id))
    #         target_path = f"{image_id}_seg.npy"
    #         _target_paths.append(target_path)
    #         np.save(target_path, binary_masks[index][0])
    
    # df["seg_paths"] = _target_paths
    torch.cuda.empty_cache()
    
    return mask_ratio

In [11]:
seg_model = smp.Unet(
    encoder_name=CFG.seg_model_name,
    encoder_weights=None,
    in_channels=3,
    classes=1,
    activation=None,
)

seg_model.to(device)
seg_model.eval()
seg_model.load_state_dict(torch.load(CFG.seg_model_weight))
print("seg_model_weight loaded")

seg_model_weight loaded


In [12]:
test_dataset = UBCSegDataset(tile_df, transforms=get_transforms())
test_loader = DataLoader(test_dataset, batch_size=4, 
                            num_workers=4, shuffle=False, pin_memory=True)

In [13]:
tile_df

,tile_path,image_id
0,test_tiles/1020_tile_26624_24576.png,1020
1,test_tiles/706_tile_65536_4096.png,706
2,test_tiles/706_tile_30720_6144.png,706
3,test_tiles/1660_tile_67584_16384.png,1660
4,test_tiles/281_tile_8192_6144.png,281
...,...,...
2877,test_tiles/431_tile_34816_22528.png,431
2878,test_tiles/286_tile_22528_18432.png,286
2879,test_tiles/1666_tile_0_8192.png,1666
2880,test_tiles/66_tile_40960_14336.png,66


In [49]:
mask_ratio = seg_infer(test_loader=test_loader, model=seg_model)

In [50]:
tile_df["mask_ratio"] = mask_ratio

In [52]:
# tile_df.groupby("image_id")["mask_ratio"].max()

image_id
1020    1.000000
1080    1.000000
1101    1.000000
1252    1.000000
1289    0.991673
1295    1.000000
1660    1.000000
1666    1.000000
1774    1.000000
1925    1.000000
1943    0.995358
1952    1.000000
281     0.893536
286     1.000000
4       1.000000
431     1.000000
66      1.000000
706     1.000000
91      0.268433
970     1.000000
Name: mask_ratio, dtype: float64

In [30]:
# class UBCSegDataset(Dataset):
#     def __init__(self, img_paths, transforms=None):
#         self.img_paths = img_paths
#         self.transforms = transforms
        
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, index):
        
#         img_path = self.img_paths[index]

#         img = cv2.imread(str(img_path))
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
#         # image_id = self.image_ids[index]
        
#         if self.transforms:
#             img = self.transforms(image=img)["image"]
            
#         return {
#             "image": img,
#         }
    

# def seg_infer(df):
    
#     _image_ids = []
#     _target_paths = []
    
#     data_transforms = {
#         "valid": A.Compose([
#             A.Resize(CFG.img_size, CFG.img_size),
#             A.Normalize(
#                     mean=[0.485, 0.456, 0.406], 
#                     std=[0.229, 0.224, 0.225], 
#                     max_pixel_value=255.0, 
#                     p=1.0
#                 ),
#             ToTensorV2()], p=1.)
#     }

#     valid_dataset = UBCSegDataset(test_df, transforms=data_transforms["valid"])

#     valid_loader = DataLoader(
#         valid_dataset,
#         batch_size=CFG.batch_size,
#         shuffle=False,
#         num_workers=CFG.num_workers,
#         pin_memory=True,
#         drop_last=False,
#     )
    
#     model_name = "efficientnet-b0"
#     model_weight = "exp-seg/seg-fold-0.pth"
    
#     model = smp.Unet(
#         encoder_name=model_name,
#         encoder_weights=None,
#         in_channels=3,
#         classes=1,
#         activation=None,
#     )

    
#     model.to(device)
#     model.load_state_dict(torch.load(model_weight))
#     print("weights loaded")

#     for index, data in enumerate(valid_loader):
#         images = data["image"].to(device)
#         image_ids = data["image_id"]
        
#         with torch.no_grad():
#             outputs = model(images)

#         probs = torch.sigmoid(outputs)
#         probs = probs.detach().cpu().numpy()

#         threshold = 0.50
#         binary_masks = (probs > threshold).astype(int)
#         for index, image_id in enumerate(image_ids):
#             _image_ids.append(str(image_id))
#             target_path = f"infer_seg/{image_id}_seg.npy"
#             _target_paths.append(target_path)
#             np.save(target_path, binary_masks[index][0])
    
# #     rst = pd.DataFrame()
# #     rst["image_id"] = _image_ids
#     df["seg_paths"] = _target_paths
    
#     return df

In [31]:
# def process_image(row):
#     image_id = row.image_id
#     image_path = row.file_path
#     is_tma = row.is_tma

#     img = cv2.imread(str(image_path))

#     height, width = img.shape[:2]
    
#     img_target_path = Path("tiles")
#     img_target_path.mkdir(parents=True, exist_ok=True)
#     tile_size = 2048
#     rows = height // tile_size
#     cols = width // tile_size

#     if is_tma:
#         print("is tma")
#     else:
#         for i in range(rows):
#             for j in range(cols):                     
#                 tile = img[
#                     i * tile_size: (i + 1) * tile_size,
#                     j * tile_size: (j + 1) * tile_size,
#                 ]
 
#                 tile_filename = f"{image_id}_{i}_{j}.png"
#                 tile_path = img_target_path / tile_filename
#                 cv2.imwrite(str(tile_path), tile)

#                 del tile
#                 gc.collect()

In [32]:
# for index, row in test_df.iterrows():

#     print(row.file_path)
#     process_image(row)
    
#     break

dataset/train_images/6175.png


Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/connection.py", line 360, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shigengtian/anaconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/shigengtian/anaconda3/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times


In [ ]:
tile_files = sorted(glob(str("tiles/*.png")))
tile_files

['tiles/6175_0_0.png',
 'tiles/6175_0_1.png',
 'tiles/6175_0_10.png',
 'tiles/6175_0_11.png',
 'tiles/6175_0_12.png',
 'tiles/6175_0_13.png',
 'tiles/6175_0_14.png',
 'tiles/6175_0_15.png',
 'tiles/6175_0_16.png',
 'tiles/6175_0_17.png',
 'tiles/6175_0_18.png',
 'tiles/6175_0_19.png',
 'tiles/6175_0_2.png',
 'tiles/6175_0_20.png',
 'tiles/6175_0_21.png',
 'tiles/6175_0_22.png',
 'tiles/6175_0_23.png',
 'tiles/6175_0_24.png',
 'tiles/6175_0_25.png',
 'tiles/6175_0_26.png',
 'tiles/6175_0_27.png',
 'tiles/6175_0_28.png',
 'tiles/6175_0_3.png',
 'tiles/6175_0_4.png',
 'tiles/6175_0_5.png',
 'tiles/6175_0_6.png',
 'tiles/6175_0_7.png',
 'tiles/6175_0_8.png',
 'tiles/6175_0_9.png',
 'tiles/6175_10_0.png',
 'tiles/6175_10_1.png',
 'tiles/6175_10_10.png',
 'tiles/6175_10_11.png',
 'tiles/6175_10_12.png',
 'tiles/6175_10_13.png',
 'tiles/6175_10_14.png',
 'tiles/6175_10_15.png',
 'tiles/6175_10_16.png',
 'tiles/6175_10_17.png',
 'tiles/6175_10_18.png',
 'tiles/6175_10_19.png',
 'tiles/6175_10_

In [ ]:
data_transforms = {
    "valid": A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}


valid_dataset = UBCSegDataset(tile_files, transforms=data_transforms["valid"])

valid_loader = DataLoader(
    valid_dataset,
    batch_size=CFG.batch_size,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False,
)

model_name = "efficientnet-b0"
model_weight = "exp-seg-2/seg-fold-0.pth"

model = smp.Unet(
    encoder_name=model_name,
    encoder_weights=None,
    in_channels=3,
    classes=1,
    activation=None,
)

    
model.to(device)
model.load_state_dict(torch.load(model_weight))
print("weights loaded")

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/connection.py", line 177, in close
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/connection.py", line 360, in _close
    self._close()
  File "/home/shigengtian/anaconda3/lib/python3.11/multiprocessing/connection.py", line 360, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shigengt

weights loaded


In [ ]:
for index, data in enumerate(valid_loader):
    image = data["image"].to(device)
    pred = model(image)
    pred = torch.sigmoid(pred)
    preds = pred.detach().cpu().numpy()
            
    threshold = 0.33
    binary_masks = (preds > threshold).astype(int)
    break

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/shigengtian/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2286052/2060729563.py", line 1, in <module>
    for index, data in enumerate(valid_loader):
                       ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/shigengtian/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 441, in __iter__
    return self._get_iterator()
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/shigengtian/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 388, in _get_iterator
    return _MultiProcessingDataLoaderIter(self)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/shigengtian/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1084, in __init__
    self._reset(loader, first_iter=True)
  File "/home/shigengtian/anaconda3/lib/python3.11/site-pac

In [43]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'
    
    
class UBCModel(nn.Module):
    def __init__(self, model_name, num_classes=5, pretrained=False, checkpoint_path=None):
        super(UBCModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)

        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)
        return output
    
class UBCClcDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df["tile_file_paths"].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            img = self.transforms(image=img)["image"]

        return {"image": img}

In [44]:
def infer_clc(tile_df, model):


    def get_transforms():
        return A.Compose(
            [
                A.Resize(CFG.img_size, CFG.img_size),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(),
            ],
            p=1.0,
        )

    test_dataset = UBCClcDataset(tile_df, transforms=get_transforms())
    test_loader = DataLoader(test_dataset, batch_size=4, 
                              num_workers=4, shuffle=False, pin_memory=True)
    probs = []
    for index, data in enumerate(test_loader):
        data = data["image"].to(device)
        outputs = model(data)
        p = torch.sigmoid(outputs)
        p = p.detach().cpu().numpy()
        probs.append(p)
        
        del data
        del outputs
        
        torch.cuda.empty_cache()
    probs = np.concatenate(probs)
    probs = np.mean(probs,0)
    max_index = np.argmax(probs)
    
    label_dict = {
        0 : "CC",
        1 : "EC",
        2 : "HGSC",
        3 : "LGSC",
        4 : "MC"
    }
    
    max_prob = probs[max_index]

    if max_prob > 0.25:
        pred_class = label_dict[max_index]
    else:
        pred_class = "Other"
    return pred_class

In [ ]:
tile_size = 2048

clc_weight = "exp-06/fold-0.pth"
model = UBCModel("tf_efficientnetv2_s_in21ft1k")
model.to(device)
model.load_state_dict(torch.load(clc_weight))
print("weights loaded")
rst_image_ids = []
rst_labels=[]
for index, row in test_df.iterrows():
    save_path = "tiles"

    os.makedirs(save_path, exist_ok=True)
    
    _img_ids = []
    _tile_file_paths = []
    
    image_id = row["image_id"]
    img_path = row["file_path"]
    mask_path = row["seg_paths"]
    is_tma = row["is_tma"]
    
    
    img = cv2.imread(str(img_path))
    
    height, width = img.shape[:2]
    mask = np.load(str(mask_path))
    mask = cv2.resize(mask, (width, height),interpolation=cv2.INTER_NEAREST)
    
    rows = height // tile_size
    cols = width // tile_size
    print(is_tma)
#     if is_tma:
#         tile = cv2.imread(img_path)
#         tile = cv2.resize(tile, (512, 512), interpolation=cv2.INTER_NEAREST)
#         tile_filename = f"{image_id}.png"
#         tile_path = f"{save_path}/{tile_filename}"
#         cv2.imwrite(tile_path, tile)
#         _img_ids.append(image_id)
#         _tile_file_paths.append(tile_path)
#         del tile
#         gc.collect()
#         pass
#     else:
#         for i in range(rows):
#             for j in range(cols):
#                 tile_mask = mask[
#                     i * tile_size : (i + 1) * tile_size,
#                     j * tile_size : (j + 1) * tile_size,
#                 ]
#                 true_percentage = tile_mask.sum() / tile_mask.size
#                 if true_percentage < 0.5:
#                     continue

#                 tile = img[
#                     i * tile_size : (i + 1) * tile_size,
#                     j * tile_size : (j + 1) * tile_size,
#                 ]
#                 tile = cv2.resize(tile, (512, 512), interpolation=cv2.INTER_NEAREST)

#                 tile_filename = f"{image_id}_{i}_{j}.png"
#                 tile_path = f"{save_path}/{tile_filename}"
#                 cv2.imwrite(tile_path, tile)
#                 _img_ids.append(image_id)
#                 _tile_file_paths.append(tile_path)
#                 del tile
#                 gc.collect()
                
                

#     del img
#     del mask
#     gc.collect()
    
#     tile_df = pd.DataFrame()
#     tile_df["image_id"] = _img_ids
#     tile_df["tile_file_paths"] = _tile_file_paths
#     print(tile_df)
#     pred_class = infer_clc(tile_df, model)
#     rst_image_ids.append(int(image_id))
#     rst_labels.append(pred_class)
#     shutil.rmtree(save_path)

/home/shigengtian/anaconda3/lib/python3.11/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21ft1k to current tf_efficientnetv2_s.in21k_ft_in1k.
  model = create_fn(


weights loaded
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
F